In [1]:
import os
from uberduck_ml_dev.data.get import get_mels, get_pitches
from uberduck_ml_dev.data.data import RADTTS_DEFAULTS as data_config

from uberduck_ml_dev.data.get import get
import librosa
import numpy as np
from scipy.io.wavfile import write
from datetime import datetime

ImportError: cannot import name 'get' from 'uberduck_ml_dev.data.get' (/usr/src/app/uberduck_ml_dev/uberduck_ml_dev/data/get.py)

In [3]:
# data_dir = "/path/to/data"
data_dir = "/usr/src/app/uberduck_ml_exp/lj/LJSpeech/"




rel_path_list = os.listdir(os.path.join(data_dir, 'wavs'))
local_path_list = [os.path.join(data_dir, 'wavs', rel_path) for rel_path in rel_path_list]
resampled_normalized_path_list = [os.path.join(data_dir, 
                                               local_path.split('.wav')[0],
                                               'audio_resampledT_normalized32768T.wav') 
                                  for local_path in local_path_list]
spectrogram_path_list = [os.path.join(data_dir, local_path.split('.wav')[0],
                                               'spectrogram.pt') 
                                  for local_path in local_path_list]


In [12]:
import os

# def find_rel_paths(directory, filename):
#     for root, dirs, files in os.walk(directory):
#         if filename in files:
#             yield os.path.relpath(os.path.join(root, filename), directory)

directory = '/usr/src/app/uberduck_ml_exp/lj/LJSpeech/'  # replace with your directory
filename = 'gt.wav'  # replace with your filename

for rel_path in find_rel_paths(directory, filename):
    print(rel_path)

In [13]:
list(find_rel_paths(directory, filename))
list(os.walk(directory))
rel_paths = list(find_rel_paths(directory, filename))
rel_paths




[]

In [ ]:
print("resample and normalize")
MAX_WAV_VALUE = 32768
sr = 22050
loading_function = lambda filename : librosa.load(filename, sr = 22050)[0]
function_ = lambda x : np.asarray((x / np.abs(x).max()) * (MAX_WAV_VALUE - 1), dtype = np.int16)
saving_function = lambda data, filename  : write(filename, 22050, data) # must be in this order

print(datetime.now())
get(function_, loading_function, saving_function, local_path_list, resampled_normalized_path_list, False)
print(datetime.now())

In [ ]:
print("compute pitches")
get_pitches(resampled_normalized_path_list, data_config, folder_path_list, method = 'radtts')

In [ ]:
print("compute mels")
get_mels(resampled_normalized_path_list, data_config, spectrogram_path_list)